In [ ]:
#%pip install pandas
#%pip install openpyxl
%pip install networkx

In [5]:
import pandas as pd
import networkx as nx

In [6]:
# Definiciones globales

%run ../Lib/pandas_options.py
%run ../Lib/define_constantes.py

In [8]:
#leer archivo plano del "DataLake"

archivo_datos_tramos = f'{DIRECTORIO_DATOS_CRUDOS}/DATOS_TRAMOS_GRAFO.csv'
tuberias_df = pd.read_csv(archivo_datos_tramos, low_memory=False)

In [9]:
tuberias_df.shape

(178097, 6)

In [10]:
tuberias_df.head()

,G3E_FID,IPID,NODO1_ID,NODO2_ID,LONGITUD,ARRANQUE
0,10319871,10319871,9566618,9416102,50.27,NO
1,10205159,10205159,9504506,9505087,4.25,NO
2,9129899,9129899,9079730,9079731,8.69,NO
3,9172395,9172395,6059291,6059297,31.82,NO
4,10205167,10205167,9504524,6049839,11.97,NO


In [11]:
# Crear un grafo dirigido
G = nx.DiGraph()

In [12]:
# Agregar todas las tuberías al grafo
for _, row in tuberias_df.iterrows():
    G.add_edge(
        row['NODO1_ID'],
        row['NODO2_ID'],
        G3E_FID=row['G3E_FID'],
        length=row['LONGITUD'],
        arranque=row['ARRANQUE']
    )


In [13]:
# Función para calcular la longitud de las redes aguas arriba para un nodo específico
def calcular_longitud_aguas_arriba(nodo, grafo):
    # Lista para almacenar los nodos visitados y la longitud total
    visitados = set()
    longitud_total = 0

    # Función recursiva para recorrer los nodos aguas arriba
    def recorrer_aguas_arriba(nodo_actual):
        nonlocal longitud_total
        visitados.add(nodo_actual)

        # Recorrer todos los predecesores (nodos aguas arriba)
        for predecesor in grafo.predecessors(nodo_actual):
            if predecesor not in visitados:
                # Sumar la longitud de la tubería que conecta el predecesor con el nodo actual
                longitud_total += grafo[predecesor][nodo_actual]['length']

                # Verificar si la arista tiene arranque = 'SI' y detener el recorrido en esa ruta
                if grafo[predecesor][nodo_actual]['arranque'] == 'SI':
                    continue  # Detener el recorrido por este camino, pero ya se sumó la longitud

                # Recursión para seguir buscando más arriba
                recorrer_aguas_arriba(predecesor)

    # Iniciar el recorrido desde el nodo especificado
    recorrer_aguas_arriba(nodo)

    return longitud_total


In [14]:
# Crear una nueva columna en el DataFrame para almacenar la longitud aguas arriba de cada tubería
tuberias_df['LONGITUD_AGUAS_ARRIBA'] = tuberias_df['NODO1_ID'].apply(lambda nodo: calcular_longitud_aguas_arriba(nodo, G))

In [15]:
#Asignar longitud_aguas_arriba = 0 para los arranques.
tuberias_df.loc[tuberias_df['ARRANQUE'] == 'SI', 'LONGITUD_AGUAS_ARRIBA'] = 0

In [16]:
tuberias_df.head()

,G3E_FID,IPID,NODO1_ID,NODO2_ID,LONGITUD,ARRANQUE,LONGITUD_AGUAS_ARRIBA
0,10319871,10319871,9566618,9416102,50.27,NO,96.32
1,10205159,10205159,9504506,9505087,4.25,NO,964.06
2,9129899,9129899,9079730,9079731,8.69,NO,126.44
3,9172395,9172395,6059291,6059297,31.82,NO,171.98
4,10205167,10205167,9504524,6049839,11.97,NO,323.02


In [17]:
# Ruta donde deseas guardar el archivo
archivo_datos_long_aguas_arr = f'{DIRECTORIO_DATOS_CRUDOS}/DATOS_LONGITUD_AGUAS_ARRIBA.csv'

In [18]:

# Guardar el DataFrame como un archivo CSV
tuberias_df.to_csv(archivo_datos_long_aguas_arr, index=False)